# 1. Import libraries and download datasets

In [ ]:
!pip install gdown -q
!apt-get install unrar -q

Reading package lists...
Building dependency tree...
Reading state information...
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


In [ ]:
import albumentations
import torch
import numpy as np
from PIL import Image
from PIL import ImageFile
from tqdm import tqdm
from torch import nn
from torch.nn.utils.rnn import pad_sequence
from torch.nn import functional as F
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics
import os
import glob
import json
import math

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# get the datasetg
!gdown 'https://drive.google.com/uc?id=1tGbBppVTgktDPcMsAZGsNuNJC_NzpZGr' -O train_set.rar
!gdown 'https://drive.google.com/uc?id=1iDYqr2yzak9u9hwf8XXC5idpSOKe4-N0' -O test_set.rar
# Unzip the downloaded dataset
!unrar x -r train_set.rar
!unrar x -r test_set.rar -Y

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Extracting  train_set/imgs/input_3_5622.png                               90%  OK 
Extracting  train_set/imgs/input_3_5623.png                               90%  OK 
Extracting  train_set/imgs/input_3_5624.png                               90%  OK 
Extracting  train_set/imgs/input_3_5625.png                               90%  OK 
Extracting  train_set/imgs/input_3_5626.png                               90%  OK 
Extracting  train_set/imgs/input_3_5627.png                               90%  OK 
Extracting  train_set/imgs/input_3_5628.png                               90%  OK 
Extracting  train_set/imgs/input_3_5629.png                               90%  OK 
Extracting  train_set/imgs/input_3_563.png                                90%  OK 
Extracting  train_set/imgs/input_3_5630.png                               90%  OK 
Extracting  train_set/i

# 2. Dataset

In [ ]:
class ClassificationDataset:
    def __init__(self, image_paths, targets, resize=None):
        # resize = (height, width)
        self.image_paths = image_paths
        self.targets = targets
        self.resize = resize

        # normalize wrt ImageNet stat
        mean = (0.485, 0.456, 0.406)
        std = (0.229, 0.224, 0.225)
        self.aug = albumentations.Compose(
            [
                albumentations.Normalize(
                    mean, std, max_pixel_value=255.0, always_apply=True
                )
            ]
        )

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, item):
        image = Image.open(self.image_paths[item]).convert("RGB")

        if self.resize is not None:
            image = image.resize(
                (self.resize[1], self.resize[0]), resample=Image.BILINEAR
            )

        image = np.array(image)
        augmented = self.aug(image=image)
        image = augmented["image"]
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)

        # Keep targets and image as is; conversion to tensors will be handled in the collate function
        return image, self.targets[item]

# 3. Model

In [ ]:
class CaptchaModel(nn.Module):
    def __init__(self, num_chars):
        super(CaptchaModel, self).__init__()
        self.conv_1 = nn.Conv2d(3, 128, kernel_size=(3, 6), padding=(1, 1))
        self.pool_1 = nn.MaxPool2d(kernel_size=(2, 2))
        self.conv_2 = nn.Conv2d(128, 64, kernel_size=(3, 6), padding=(1, 1))
        self.pool_2 = nn.MaxPool2d(kernel_size=(2, 2))
        self.linear_1 = nn.Linear(768, 64)
        self.drop_1 = nn.Dropout(0.2)
        # bidirectional LSTM outputs 32 features, 2 layers (due to bidirection) hence 64 in total
        self.lstm = nn.GRU(64, 32, bidirectional=True, num_layers=2, dropout=0.25, batch_first=True)
        self.output = nn.Linear(64, num_chars + 1)

    def forward(self, images, targets=None, target_lengths=None):
        bs, _, _, _ = images.size()
        x = F.relu(self.conv_1(images))
        x = self.pool_1(x)
        x = F.relu(self.conv_2(x))
        x = self.pool_2(x)
        x = x.permute(0, 3, 1, 2)
        x = x.view(bs, x.size(1), -1)
        x = F.relu(self.linear_1(x))
        x = self.drop_1(x)
        x, _ = self.lstm(x)
        x = self.output(x)
        x = x.permute(1, 0, 2)

        if targets is not None:
            log_probs = F.log_softmax(x, 2)
            input_lengths = torch.full(
                size=(bs,), fill_value=log_probs.size(0), dtype=torch.int32
            )
            loss = nn.CTCLoss(blank=0)(
                log_probs, targets, input_lengths, target_lengths
            )
            return x, loss

        return x, None

# 4. Training

### Config

In [ ]:
# Config paramters
TRAIN_DATA_DIR = "train_set/imgs/"
TRAIN_LABELS_DIR = "train_set/labels.json"
TEST_DATA_DIR = "test_set_2/imgs/"
TEST_LABELS_DIR = "test_set_2/labels.json"
BATCH_SIZE = 32
IMAGE_WIDTH = 160
IMAGE_HEIGHT = 50
NUM_WORKERS = 2
EPOCHS = 120
DEVICE = "cuda"
# DEVICE = "cpu"

### Utils

In [ ]:
# Some utils
def train_fn(model, data_loader, optimizer):
    model.train()
    fin_loss = 0
    tk0 = tqdm(data_loader, total=len(data_loader))
    for data in tk0:
        for key, value in data.items():
            data[key] = value.to(DEVICE)
        optimizer.zero_grad()
        _, loss = model(**data)
        loss.backward()
        optimizer.step()
        fin_loss += loss.item()
    return fin_loss / len(data_loader)


def eval_fn(model, data_loader):
    model.eval()
    fin_loss = 0
    fin_preds = []
    tk0 = tqdm(data_loader, total=len(data_loader))
    with torch.no_grad():
        for data in tk0:
            for key, value in data.items():
                data[key] = value.to(DEVICE)
            batch_preds, loss = model(**data)
            fin_loss += loss.item()
            fin_preds.append(batch_preds)
        return fin_preds, fin_loss / len(data_loader)

def custom_collate_fn(batch):
    # batch holds list of (image, target) pairs
    # Initialize lists to hold the images and targets
    images, target_sequences = zip(*batch)

    # Convert images to tensor
    images = torch.stack([torch.tensor(img, dtype=torch.float) for img in images], dim=0)

    # Compute lengths of each target sequence
    target_lengths = torch.tensor([len(tgt) for tgt in target_sequences], dtype=torch.int32)

    # Pad target sequences
    # Note: pad_sequence expects a list of tensors, so ensure targets are tensors
    targets = pad_sequence([torch.tensor(tgt, dtype=torch.long) for tgt in target_sequences],
                           batch_first=True, padding_value=0)

    return {
        "images": images,
        "targets": targets,
        "target_lengths": target_lengths
    }

def remove_duplicates(x):
    if len(x) < 2:
        return x
    fin = ""
    for j in x:
        if fin == "":
            fin = j
        else:
            if j == fin[-1]:
                continue
            else:
                fin = fin + j
    return fin

def decode_predictions(preds, encoder):
    preds = preds.permute(1, 0, 2)
    preds = torch.softmax(preds, 2)
    preds = torch.argmax(preds, 2)
    preds = preds.detach().cpu().numpy()
    cap_preds = []
    for j in range(preds.shape[0]):
        temp = []
        for k in preds[j, :]:
            k = k - 1
            if k == -1:
                temp.append("§")
            else:
                p = encoder.inverse_transform([k])[0]
                temp.append(p)
        tp = "".join(temp).replace("§", "")
        cap_preds.append(remove_duplicates(tp))
    return cap_preds

def load_data(input_path, label_path):
    image_files = glob.glob(os.path.join(input_path, "*.png"))
    labels_dict = json.load(open(label_path, 'r'))
    targets_orig = [labels_dict[os.path.basename(x)] for x in image_files]
    targets = [list(x) for x in targets_orig]
    targets_flat = [c for clist in targets for c in clist]
    return targets_orig, targets, targets_flat, image_files

def compute_model_stat(model, data_loader, lbl_enc, targets_orig, verbose=True):
    valid_preds, loss = eval_fn(model, data_loader)
    valid_captcha_preds = []
    for vp in valid_preds:
        current_preds = decode_predictions(vp, lbl_enc)
        valid_captcha_preds.extend(current_preds)

    if verbose:
        combined = list(zip(targets_orig, valid_captcha_preds))
        print(combined[:10])

    # val_dup_rem = [remove_duplicates(c) for c in val_targets_orig]
    # val_accuracy = metrics.accuracy_score(val_dup_rem, valid_captcha_preds)
    accuracy = metrics.accuracy_score(targets_orig, valid_captcha_preds)
    return loss, accuracy

def encode_CTC_labels(encoder, data):
    return np.array([encoder.transform(x) for x in data]) + 1

### Load data

In [ ]:
(
    targets_orig,
    targets,
    targets_flat,
    image_files
) = load_data(TRAIN_DATA_DIR, TRAIN_LABELS_DIR)
(
    test_targets_orig,
    test_targets,
    test_targets_flat,
    test_imgs
) = load_data(TEST_DATA_DIR, TEST_LABELS_DIR)

lbl_enc = preprocessing.LabelEncoder()
lbl_enc.fit(targets_flat)
targets_enc = encode_CTC_labels(lbl_enc, targets)
test_targets_enc = encode_CTC_labels(lbl_enc, test_targets)

(
    train_imgs,
    val_imgs,
    train_targets, # encoded
    val_targets, # encoded
    _,
    val_targets_orig,
) = model_selection.train_test_split(
    image_files, targets_enc, targets_orig, test_size=0.1, random_state=42
)

train_dataset = ClassificationDataset(
    image_paths=train_imgs,
    targets=train_targets,
    resize=(IMAGE_HEIGHT, IMAGE_WIDTH),
)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=True,
    collate_fn=custom_collate_fn
)
val_dataset = ClassificationDataset(
    image_paths=val_imgs,
    targets=val_targets,
    resize=(IMAGE_HEIGHT, IMAGE_WIDTH),
)
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=False,
    collate_fn=custom_collate_fn
)
test_dataset = ClassificationDataset(
    image_paths=test_imgs,
    targets=test_targets_enc,
    resize=(IMAGE_HEIGHT, IMAGE_WIDTH),
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=False,
    collate_fn=custom_collate_fn
)

<ipython-input-7-0c817027a862>:111: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([encoder.transform(x) for x in data]) + 1
<ipython-input-7-0c817027a862>:111: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([encoder.transform(x) for x in data]) + 1


### Train

In [ ]:
model = CaptchaModel(num_chars=len(lbl_enc.classes_))
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.8, patience=5, verbose=True
)

best_acc = -math.inf
for epoch in range(EPOCHS):
    train_loss = train_fn(model, train_loader, optimizer)

    # validate the model
    val_loss, val_acc = compute_model_stat(
                            model,
                            val_loader,
                            lbl_enc,
                            val_targets_orig
                        )
    stat = f"Epoch={epoch}, Train Loss={train_loss}, Val Loss={val_loss}, Val Acc={val_acc}"

    # test every 5 epoch
    if epoch % 5 == 4:
        test_loss, test_acc = compute_model_stat(
                                model,
                                test_loader,
                                lbl_enc,
                                test_targets_orig
                            )
        stat += f", Test Loss={test_loss}, Test Acc={test_acc}"

    print(stat) # print the stat out

    # save model if best
    if val_acc > best_acc:
        model_path = "model.pth"
        torch.save(model.state_dict(), model_path)
        best_acc = val_acc

    scheduler.step(val_loss) # schedule lr

100%|██████████| 157/157 [00:04<00:00, 38.85it/s]


[('BOUH', ''), ('HLWU', ''), ('NLBO', ''), ('IYRM', ''), ('WRBY', ''), ('EWPW', ''), ('KVDZ', ''), ('RBW', ''), ('RLZO', ''), ('OXG', '')]
Epoch=0, Train Loss=3.7850604984212954, Val Loss=3.5363505266274617, Val Acc=0.0


100%|██████████| 157/157 [00:04<00:00, 32.24it/s]


[('BOUH', ''), ('HLWU', ''), ('NLBO', ''), ('IYRM', ''), ('WRBY', ''), ('EWPW', ''), ('KVDZ', ''), ('RBW', ''), ('RLZO', ''), ('OXG', '')]
Epoch=1, Train Loss=3.5016594853228344, Val Loss=3.4699127628545092, Val Acc=0.0


100%|██████████| 157/157 [00:04<00:00, 31.86it/s]


...

100%|██████████| 157/157 [00:04<00:00, 37.34it/s]


[('BOUH', 'BOUH'), ('HLWU', 'HLWU'), ('NLBO', 'NLBO'), ('IYRM', 'IYRM'), ('WRBY', 'WRBY'), ('EWPW', 'EWPW'), ('KVDZ', 'KVDZ'), ('RBW', 'RBW'), ('RLZO', 'RLZO'), ('OXG', 'OXG')]


100%|██████████| 4/4 [00:00<00:00, 17.52it/s]


[('BAHO', 'BAHO'), ('AWOF', 'AWOF'), ('DEFE', 'OEFE'), ('GOFY', 'GOFY'), ('FUTYH', 'FUTYK'), ('FUWY', 'FLUVNY'), ('OBIR', 'CVIA'), ('ODOW', 'ODVN'), ('LEXE', 'LEXE'), ('UBOHI', 'UBOHI')]
Epoch=109, Train Loss=0.031960344414492144, Val Loss=0.05245281609404049, Val Acc=0.829, Test Loss=1.3191289603710175, Test Acc=0.4365079365079365


 28%|██▊       | 44/157 [00:01<00:02, 41.59it/s]

In [ ]:
while True:
    pass